## Import Libraries

In [28]:
import configparser
import os

import logging
import boto3
from botocore.exceptions import ClientError

import pandas as pd

from pyspark.sql import SparkSession
from pyspark import SparkContext

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

## Make sure you have an AWS secret and access key

- Create a new IAM user in your AWS account
- Give it AdministratorAccess, From Attach existing policies directly Tab
- Take note of the access key and secret
- Edit the file dwh.cfg in the same folder as this notebook and fill
<font color='red'>
<BR>
[AWS]<BR>
KEY= YOUR_AWS_KEY<BR>
SECRET= YOUR_AWS_SECRET<BR>

## Read the Configuration file

In [3]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

## Read the AWS User Access Key ID and Secret Key

In [4]:
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

KEY= config['AWS']['AWS_ACCESS_KEY_ID']
SECRET= config['AWS']['AWS_SECRET_ACCESS_KEY']

## Create a Local Spark Session

In [5]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [6]:
spark = create_spark_session()

In [7]:
spark

## Read the Song data JSON file stored in S3 bucket

In [8]:
INPUT_DATA = config['S3']['SONG_DATA']

song_data = INPUT_DATA + "/A/B/C/*.json"
song_data

's3a://udacity-dend/song_data/A/B/C/*.json'

## Read the JSON file into a Spark DataFrame

In [9]:

song_f1 = spark.read.json(song_data)
song_f1.limit(2).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARLTWXK1187FB5A3F8,32.74863,"Fort Worth, TX",-97.32925,King Curtis,326.00771,1,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),0
1,ARIOZCU1187FB3A3DC,NaN,"Hamlet, NC",NaN,JOHN COLTRANE,220.44689,1,SOCEMJV12A6D4F7667,Giant Steps (Alternate Version_ Take 5_ Altern...,0


## Read the User log data JSON file stored in S3 bucket

In [10]:
INPUT_DATA = config['S3']['LOG_DATA']

log_data = INPUT_DATA + "/2018/11/*.json"
log_data

's3a://udacity-dend/log_data/2018/11/*.json'

In [11]:
log_f1 = spark.read.json(log_data)
log_f1.limit(2).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


## Read the DataWareHouse Configuration details to create a Redshift Cluster

In [53]:
DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_ENDPOINT           = config.get("DWH", "DWH_ENDPOINT")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")
DWH_ROLE_ARN           = config.get("IAM_ROLE", "ARN")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

'''
pd.DataFrame({"Param":
["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME", "DWH_ROLE_ARN"],
              "Value":
[DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME, DWH_ROLE_ARN]
             })
'''

'\npd.DataFrame({"Param":\n["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME", "DWH_ROLE_ARN"],\n              "Value":\n[DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME, DWH_ROLE_ARN]\n             })\n'

## Create clients for EC2, S3, IAM, and RedShift

In [43]:
ec2 = boto3.resource("ec2", 
                     region_name="us-east-1",
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET)

s3 = boto3.resource("s3",
                    region_name="us-east-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

iam = boto3.client("iam",
                    region_name="us-east-1",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

redshift = boto3.client("redshift",
                          region_name="us-east-1",
                          aws_access_key_id=KEY,
                          aws_secret_access_key=SECRET)

## Filter the columns from the songs data dataframe to create songs table

### Schema of the DataFrame

In [16]:
song_f1.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



- songs table: columns
- song_id, title, artist_id, year, duration

In [20]:
songs = song_f1.select(["song_id", "title", "artist_id", "year", "duration"]).dropDuplicates()
songs.show()

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOQFYBD12AB0182188|               Intro|ARAADXM1187FB3ECDB|1999| 67.63057|
|SOFIUVJ12A8C13C296|Will You Tell Me ...|AR9OEB71187B9A97C6|2005|397.16526|
|SOGDBUF12A8C140FAA|               Intro|AR558FS1187FB45658|2003| 75.67628|
|SOBHXUU12A6D4F5F14|National Emblem (...|ARBDJHO1252CCFA6FC|   0|188.73424|
|SOIKLJM12A8C136355|           Eso Duele|AR7AE0W1187B98E40E|2003|196.25751|
|SOEVPWF12A58A7D254|         Astral eyes|ARCJWLU1187B9ADD36|1997|274.54649|
|SODWBIK12AB017F87D|               Aüita|ARSMG8X1187B99CA99|2009| 210.1024|
|SOUPIRU12A6D4FA1E1| Der Kleine Dompfaff|ARJIE2Y1187B994AB7|   0|152.92036|
|SOHTCZS12A6D4FC402|  The Christmas Song|AROSPS51187B9B481F|1965|197.95546|
|SODAUVL12A8C13D184|           Prognosis|ARWB3G61187FB49404|2000|363.85914|
|SOYLILV12A8

- artists table: columns
- artist_id, name, location, lattitude, longitude

In [22]:
artists = song_f1.select(["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]).dropDuplicates()
artists.show()

+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|AR0IAWL1187B9A96D0|        Danilo Perez|              Panama|         8.4177|       -80.11278|
|ARWB3G61187FB49404|         Steve Morse|      Hamilton, Ohio|           null|            null|
|ARJIE2Y1187B994AB7|         Line Renaud|                    |           null|            null|
|ARVBRGZ1187FB4675A|        Gwen Stefani|                    |           null|            null|
|ARCKOJF1241B9C75B4|        Eddie Sierra|                    |           null|            null|
|ARLTWXK1187FB5A3F8|         King Curtis|      Fort Worth, TX|       32.74863|       -97.32925|
|AR5S9OB1187B9931E3|         Bullet Boys|     Los Angeles, CA|       34.05349|      -118.24532|
|ARPFHN61187FB575F6|         Lupe Fiasco

In [25]:
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", KEY)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", SECRET)

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3n.endpoint", "s3.amazonaws.com")

## Create a S3 Bucket for a given region

In [29]:
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

### Create a bucket to store the dimension tables

In [41]:
S3 = create_bucket("deshagvrk")

In [54]:
location = boto3.client('s3').get_bucket_location(Bucket="deshagvrk")['LocationConstraint']
url = "https://s3-%s.amazonaws.com/%s/%s" % (location, "deshagvrk", key)

In [56]:
url

'https://s3-None.amazonaws.com/deshagvrk/upload-file'

## Save the songs DataFrame (table) in Parquet format in S3 bucket

In [50]:
spark.conf.set("spark.sql.parquet.compression.codec", "gzip")

In [52]:
songs.write.parquet("s3a://deshagvrk/parquet/songs.parquet")

Py4JJavaError: An error occurred while calling o232.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOpera